#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [114]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['kickstarter']

In [115]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

C:\Users\basti\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [42]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [116]:
#Tri et Netoyage de la base
df_ks = df_ks.drop(["currency","deadline","goal","backers"],axis=1)
df_ks = df_ks.dropna()
df_ks = df_ks.drop_duplicates()
df_ks["name"] = df_ks["name"].str.replace("(Canceled)"," ")
df_ks = df_ks.rename(columns={"usd pledged" : "usd_pledged"})




In [117]:
df_ks.head()

,ID,name,category,main_category,launched,pledged,state,country,usd_pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" ( )",Rock,Music,2011-08-17 06:31:31,1145,canceled,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,2012-07-23 20:46:48,1851,failed,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,2016-07-01 21:55:54,7534,successful,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,2017-08-28 15:47:02,16298,successful,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,2016-01-25 17:37:10,1,failed,CA,0.708148,0.738225


In [118]:
#Retyping
pd.to_numeric([df_ks["usd_pledged"],df_ks["pledged"]],errors="coerce")
df_ks.dropna()



,ID,name,category,main_category,launched,pledged,state,country,usd_pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" ( )",Rock,Music,2011-08-17 06:31:31,1145,canceled,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,2012-07-23 20:46:48,1851,failed,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,2016-07-01 21:55:54,7534,successful,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,2017-08-28 15:47:02,16298,successful,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,2016-01-25 17:37:10,1,failed,CA,0.708148,0.738225
...,...,...,...,...,...,...,...,...,...,...
149995,574841011,Scutes Headphones - A new way to listen to you...,Product Design,Design,2014-10-14 20:44:37,3664,canceled,US,3664,3664.000000
149996,235190237,My Drunk @$$ Reviews,Webseries,Film & Video,2012-04-18 04:42:47,0,failed,US,0,0.000000
149997,758540498,Freedom Planet - High Speed Platform Game,Video Games,Games,2013-01-16 00:53:32,25472.7,successful,US,25472.7,25472.690000
149998,1624952469,Clown vs Mime,Webseries,Film & Video,2011-04-27 01:22:52,76,failed,US,76,76.000000


### Importer les données

In [120]:
DOCUMENTS = df_ks.to_dict("records")
collection.insert_many(DOCUMENTS)

In [121]:
database.list_collection_names()


['kickstarter']

## Question 1  

In [127]:
five_max = collection.find().sort([("pledged",-1)]).limit(5)

for i in five_max:
    print(i)

{'_id': ObjectId('61af67a8f7a5cbe2d7478b96'), 'ID': 218218259, 'name': 'True North ï¿½ Repurposed Sailcloth Products', 'category': 'Product Design', 'main_category': 'Design', 'launched': '2016-05-10 15:53:56', 'pledged': '9996.0', 'state': 'successful', 'country': 'US', 'usd_pledged': '9996.0', 'usd_pledged_real': 9996.0}
{'_id': ObjectId('61af67a8f7a5cbe2d7471346'), 'ID': 1742823215, 'name': 'Polco: A Digital Platform for Better Civic Communication', 'category': 'Web', 'main_category': 'Technology', 'launched': '2017-06-27 21:46:04', 'pledged': '9994.0', 'state': 'failed', 'country': 'US', 'usd_pledged': '876.0', 'usd_pledged_real': 9994.0}
{'_id': ObjectId('61af67a8f7a5cbe2d746f95a'), 'ID': 1247285954, 'name': '2016 Gaming Calendar', 'category': 'Tabletop Games', 'main_category': 'Games', 'launched': '2015-06-26 14:57:14', 'pledged': '9994.0', 'state': 'successful', 'country': 'US', 'usd_pledged': '9994.0', 'usd_pledged_real': 9994.0}
{'_id': ObjectId('61af67a8f7a5cbe2d7472c78'), 'I

## Question 2

In [132]:
success = collection.aggregate([{"$group" : {"_id":"$state", "state": {"$sum":1}}}])
list(success)

[{'_id': 'successful', 'state': 52998},
 {'_id': 'canceled', 'state': 15350},
 {'_id': 'live', 'state': 1132},
 {'_id': '850.0', 'state': 1},
 {'_id': 'suspended', 'state': 738},
 {'_id': 'failed', 'state': 78297}]

## Question 3

In [133]:
categories = collection.aggregate([{"$group" : {"_id":"$main_category", "main_category": {"$sum":1}}}])
list(categories)

[{'_id': 'Theater', 'main_category': 4305},
 {'_id': 'Journalism', 'main_category': 1807},
 {'_id': 'Photography', 'main_category': 4176},
 {'_id': 'Comics', 'main_category': 4139},
 {'_id': 'Art', 'main_category': 11260},
 {'_id': 'Games', 'main_category': 13987},
 {'_id': 'Publishing', 'main_category': 15721},
 {'_id': 'Indie Rock', 'main_category': 1},
 {'_id': 'Food', 'main_category': 9805},
 {'_id': 'Fashion', 'main_category': 8943},
 {'_id': 'Dance', 'main_category': 1488},
 {'_id': 'Design', 'main_category': 11911},
 {'_id': 'Film & Video', 'main_category': 24982},
 {'_id': 'Music', 'main_category': 19657},
 {'_id': 'Technology', 'main_category': 12871},
 {'_id': 'Crafts', 'main_category': 3463}]

## Question 4

## Question 5

In [136]:
more_than = collection.find({"$and":[{"usd_pledged_real":{"$gt":200000}},{"country":"US"}]},{"name":1})
list(more_than)

[{'_id': ObjectId('61af67a7f7a5cbe2d745bb0b'),
  'name': 'The uKeg Pressurized Growler for Fresh Beer'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bb9a'),
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bc47'),
  'name': 'Legion Solar - A Better Way to Energy Independence'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bd64'),
  'name': 'Edyn: Welcome to the connected garden.'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bd73'), 'name': 'Period Panties'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745be27'),
  'name': 'Smartkase for iPhone 7 - Dual Sim +Memory Card +Battery Case'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745be5a'), 'name': 'Shadowrun: Hong Kong'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bed0'),
  'name': 'Kurt Vonnegut: Unstuck in Time'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bf25'),
  'name': 'Battle Worlds: Kronos - Turn-based strategy revisited'},
 {'_id': ObjectId('61af67a7f7a5cbe2d745bfb8'),
  'name': 'Hie Diaper Bag: Your New Bes

## Question 6 

In [140]:
sport = collection.find({"name":{"$regrex":"Sport"}})
list(sport)


OperationFailure: unknown operator: $regrex, full error: {'ok': 0.0, 'errmsg': 'unknown operator: $regrex', 'code': 2, 'codeName': 'BadValue'}